# 监视模型

将模型作为服务部署到生产中后，你将需要对其进行监视以跟踪使用情况并浏览其处理的请求。你可以使用 Azure Application Insights 监视模型服务终结点的活动。

## 连接到工作区

首先，请连接到你的工作区。

> **备注**：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## 准备要部署的模型

现在我们需要一个要部署的模型。运行以下代码以执行下列操作：

1. 创建并注册数据集。
2. 使用数据集训练模型。
3. 注册模型。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from azureml.core import Dataset

# 将数据文件上传到默认数据存储
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-data/',
                       overwrite=True,
                       show_progress=True)

#从数据存储上的路径创建表格数据集
print('Creating dataset...')
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 注册表格数据集
print('Registering dataset...')
try:
    data_set = data_set.register(workspace=ws, 
                               name='diabetes dataset',
                               description='diabetes data',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

# 在工作区中创建 Azure ML 试验
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 加载糖尿病数据集
print("Loading Data...")
diabetes = data_set.to_pandas_dataframe()

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练决策树模型
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# 保存训练的模型
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# 完成运行
run.complete()

# 注册模型
print('Registering model...')
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 获取已注册的模型
model = ws.models['diabetes_model']

print('Model trained and registered.')

## 将模型部署为 Web 服务

现在可以将已注册的模型部署为 Web 服务。

首先，为部署配置文件创建一个文件夹

In [ ]:
import os

folder_name = 'diabetes_service'

# 为 Web 服务文件创建文件夹
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# 设置评分脚本和环境文件的路径
script_file = os.path.join(experiment_folder,"score_diabetes.py")
env_file = os.path.join(experiment_folder,"diabetes_env.yml")

现在需要一个入口脚本，此服务将使用它对新数据进行评分。

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# 加载服务时调用
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# 收到请求时调用
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # Get a prediction from the model
    predictions = model.predict(np_data)
    
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

还需要一个适用于服务环境的 Conda 配置文件。

In [ ]:
%%writefile $env_file
name: inference_env
dependencies:
- python=3.6.2
- scikit-learn
- pip
- pip:
  - azureml-defaults


现在可以部署此服务（在本例中为 Azure 容器实例 (ACI)）。

> **备注**：这可能需要几分钟时间 - 等待状态显示为“**正常**”。

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig

# 配置评分环境
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

service_name = "diabetes-service-app-insights"
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aci_service = Model.deploy(workspace=ws,
                           name= service_name,
                           models= [model],
                           inference_config= inference_config,
                           deployment_config=deployment_config)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

## 启用 Application Insights

接下来需要为此服务启用 Application Insights。

In [ ]:
# 启用 AppInsights
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

## 使用 Web 服务

部署此服务后，现在可以从客户端应用程序使用它。

首先确定这些应用程序必须将其请求提交到的 URL。

In [ ]:
endpoint = aci_service.scoring_uri
print(endpoint)

你已知道终结点 URI，应用程序现在可以发出 HTTP 请求、发送 JSON（或二进制）格式的患者数据以及接收预测的类。

> **提示**：如果由于服务终结点尚未准备就绪而发生错误，请等待几秒钟，然后重试！

In [ ]:
import requests
import json

# 创建用于推理的新数据
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# 将该数组转换为 JSON 文档中的可序列化列表
input_json = json.dumps({"data": x_new})

# 设置内容类型
headers = { 'Content-Type':'application/json' }

# 获取预测
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

现在可以查看为服务终结点记录的数据：

1. 在 [Azure 门户](https://portal.azure.com)中，打开你的机器学习工作区。
2. 在“**概述**”页面上，单击相关联的 **Application Insights** 资源的链接。
3. 在 Application Insights 边栏选项卡上，单击“**日志**”。 

    > **备注**：如果这是你第一次打开日志分析，可能需要点击“**开始**”以打开查询编辑器。如果显示说明如何编写查询的提示，请关闭它。

4. 将以下查询粘贴到查询编辑器中，然后单击“**运行**”
    ```
    traces
    |where  message == "STDOUT"
      and customDimensions.["Service Name"] == "diabetes-service-app-insights"
    |project timestamp, customDimensions.Content
    ```
5. 查看结果。刚开始可能没有结果，因为 ACI Web 服务可能需要 5 分钟才能将遥测发送到 Application Insights。等待几分钟后重新运行查询，直到看到记录的数据和预测。
6. 查看记录的数据后，关闭 Application Insights 查询页面。

## 删除服务

当不再需要服务时，应将其删除。

> **备注**：如果正在使用服务，可能无法立即将其删除。

In [ ]:
try:
    aci_service.delete()
    print('Service deleted.')
except Exception as ex:
    print(ex.message)

有关使用 Application Insights 监视已部署的服务的详细信息，请参阅 [Azure 机器学习文档](https://docs.microsoft.com/azure/machine-learning/how-to-enable-app-insights)。